In [1]:
import pandas as pd

In [8]:
def merge_01( df ):

    ret = df.copy()

    row0 = ret.iloc[0].fillna(method="ffill")
    row1 = ret.iloc[1].fillna("")

    cols = []
    for i in range(len(row0)):
	if row0[i] is None:
            cols.append(str(row1[i]))
	else:
            cols.append(str(row0[i]) + "_" + str(row1[i]))

            cols = map(lambda x: x.replace("nan_",""), cols)

    ret = ret.drop(0)
    ret = ret.drop(1)

    ret.columns = cols

    def remove_trail(s):
        s = s.strip()
        if (s[-1] == "_"):
            return s[:-1]
        return s

    ret.columns = map(remove_trail, ret.columns)

    return ret.reset_index()


In [49]:
import re
def get_df():
    
    ret = pd.read_excel("raw/Gov Revised FY 18 FY 19.xlsx", header=None)
    
    ret = merge_01(ret).drop(0)
    
    cols = []
    
    for c in ret.columns:
        cols.append(re.sub(r'[^a-z0-9_]+',' ', c.lower()))
    
    ret.columns = cols
    
    pstring = ""
    for c in ret.columns:
        pstring += "'" + c + "',"
    #print pstring
    
    for c in cols[2:]:
        try:
            ret[c] = pd.to_numeric(ret[c],errors="coerce")
        except:
            pass
    
    return ret

get_df().dtypes

index                                                                             int64
grantee                                                                          object
pilot state owned real property_fy 2017                                         float64
pilot state owned real property_gov rec original fy 18                          float64
pilot state owned real property_gov rec original fy 19                          float64
pilot state owned real property_gov rec revised fy 18                           float64
pilot state owned real property_gov rec revised fy 19                           float64
pilot state owned real property                                                 float64
pilot colleges hospitals_fy 2017                                                float64
pilot colleges hospitals_gov rec original fy 18                                 float64
pilot colleges hospitals_gov rec original fy 19                                 float64
pilot colleges hospitals_gov rec

In [50]:
df = get_df()

In [51]:
df.to_csv("output/budget_changes.csv",index=False)

In [53]:
df[df["grantee"] == "Hartford"]

,index,grantee,pilot state owned real property_fy 2017,pilot state owned real property_gov rec original fy 18,pilot state owned real property_gov rec original fy 19,pilot state owned real property_gov rec revised fy 18,pilot state owned real property_gov rec revised fy 19,pilot state owned real property,pilot colleges hospitals_fy 2017,pilot colleges hospitals_gov rec original fy 18,...,cumulative total_gov rec original fy 18,cumulative total_gov rec original fy 19,cumulative total_gov rec revised fy 18,cumulative total_gov rec revised fy 19,cumulative total,cumulative total_diff fy 18 gov rec vs gov rec ii,cumulative total_diff fy 19 gov rec vs comm,cumulative total,cumulative total_diff fy 18 gov rec vs fy 17,cumulative total
64,66,Hartford,13560353.0,12189166.52,12189166.52,12189166.52,12189166.52,NaN,23664027.0,5887469.0,...,3.215654e+08,3.197213e+08,3.292819e+08,3.279922e+08,NaN,7.716529e+06,8.270955e+06,NaN,5.717579e+07,NaN


In [69]:
def get_sped_df():
    ret = pd.read_excel("raw/OPM Excess Cost 5 year average.xlsx",skiprows=8)
    
    #print ret.columns
    ret.columns = [u'Unnamed: 0', u'town', u'Unnamed: 2', u'Unnamed: 3',
       u'Unnamed: 4', u'Unnamed: 5', u'Unnamed: 6', u'sped']
    return ret[["town","sped"]]
    
get_sped_df().head()


,town,sped
0,Andover,136950
1,Ansonia,1020537
2,Ashford,172340
3,Avon,1081136
4,Barkhamsted,129710


In [88]:
def get_sped(town):
    spdf = get_sped_df()
    try:
        matches = spdf[spdf["town"].str.upper() == town.upper()]
    except:
        pass
    
    try:
        if len(matches) == 1:
            return float(matches["sped"])
    except:
        pass
    
    print "no match: " + town
    
df["special education_fy 2017"] = df["grantee"].apply(get_sped)

no match: Bantam (Bor.)
no match: Danielson (Bor.)
no match: Fenwick (Bor.)
no match: Groton (City of)
no match: Groton Long Point
no match: Jewett City (Bor.)
no match: Litchfield (Bor.)
no match: Newtown (Bor.)
no match: Stonington (Bor.)
no match: Woodmont (Bor.)
no match: District No.   1
no match: District No.   4
no match: District No.   5
no match: District No.   6
no match: District No.   7
no match: District No.   8
no match: District No. 10
no match: District No. 12
no match: District No. 13
no match: District No. 14
no match: District No. 15
no match: District No. 16
no match: District No. 17
no match: District No. 18
no match: District No. 19
no match: Education Conn
no match: EASTCONN
no match: SDE Admin Costs
no match: Blmfld Cntr FD
no match: Blmfld Blue Hills FD
no match: Cromwell FD
no match: Hazardville FD #3
no match: Enfield Thmpsnvll 
no match: Manchester - 8th Utility Dist
no match: Groton: Poq. Brdg FD
no match: Middletown City FD
no match: Middletown  South Fire

TypeError: cannot concatenate 'str' and 'float' objects

In [ ]:
df["special education_fy 2017"]

In [89]:
df.to_csv("output/budget_changes.csv",index=False)